In [1]:
!pip install torch torchvision transformers pandas scikit-learn tqdm Pillow bert-score evaluate rouge-score rapidfuzz sentence-transformers inflect accelerate


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 71.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 88.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.5 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=2493

In [2]:
from PIL import Image
import torch
from tqdm import tqdm
from peft import get_peft_model, LoraConfig, TaskType
from transformers import AutoProcessor, AutoModelForVision2Seq
import pandas as pd
from pathlib import Path
from sklearn.preprocessing import LabelEncoder
from torchvision import models, transforms
from bert_score import score
import re
import inflect
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import evaluate
from bert_score import BERTScorer
from rouge_score import rouge_scorer
from rapidfuzz.distance import Levenshtein
from sentence_transformers import SentenceTransformer, util
import numpy as np
import pandas as pd
import os
import ast
import pandas as pd
from PIL import Image
import torch
from sklearn.model_selection import train_test_split  # scikit-learn split :contentReference[oaicite:2]{index=2}
import evaluate                                        # metrics :contentReference[oaicite:3]{index=3}
from transformers import Blip2Processor, Blip2ForConditionalGeneration , BitsAndBytesConfig, get_linear_schedule_with_warmup
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, PeftModel
from transformers.data.data_collator import default_data_collator

2025-05-17 06:30:04.893730: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747463405.076118      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747463405.127679      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
train_df = pd.read_csv('/kaggle/input/datasetnewsplit/train.csv')
val_df = pd.read_csv('/kaggle/input/datasetnewsplit/val.csv')
test_df = pd.read_csv('/kaggle/input/datasetnewsplit/test.csv')
print(f"Loaded training dataset with {len(train_df)} entries")
print(f"Loaded validation dataset with {len(val_df)} entries")
print(f"Loaded testing dataset with {len(test_df)} entries")

Loaded training dataset with 80001 entries
Loaded validation dataset with 20001 entries
Loaded testing dataset with 20001 entries


In [4]:
val_df["full_image_path"] = val_df["full_image_path"].apply(lambda p: p.replace("\\", "/"))
val_df["full_image_path"] = val_df["full_image_path"].apply(lambda p: str(Path("/kaggle/input") / p))
train_df["full_image_path"] = train_df["full_image_path"].apply(lambda p: p.replace("\\", "/"))
train_df["full_image_path"] = train_df["full_image_path"].apply(lambda p: str(Path("/kaggle/input") / p))
test_df["full_image_path"] = test_df["full_image_path"].apply(lambda p: str(Path("/kaggle/input") / p))
test_df["full_image_path"] = test_df["full_image_path"].apply(lambda p: p.replace("\\", "/"))

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model_path = "ibm-granite/granite-vision-3.2-2b"
processor = AutoProcessor.from_pretrained(model_path, use_fast = True)
model = AutoModelForVision2Seq.from_pretrained(model_path).to(device)

processor_config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.64k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/2.05k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/6.97k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/777k [00:00<?, ?B/s]

...

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model-00002-of-00002.safetensors:   0%|          | 0.00/951M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [6]:
# Evaluation variables
import time
all_preds_GRANITE = []
all_actuals_GRANITE = []
correct_GRANITE = 0
total_GRANITE = 0
start_time = time.time()
TIME_LIMIT= 42000

In [7]:
import re
import inflect
inflect_engine = inflect.engine()

def normalize_answer(ans):
    ans = ans.strip().lower()
    ans = re.sub(r'[^\w\s]', '', ans)  # remove punctuation
    if ans.isdigit():
        ans = inflect_engine.number_to_words(ans)
    words = ans.split()
    return words[0] if words else ""  # return first word or empty string



In [8]:
# for idx, row in tqdm(df.iterrows(), total=len(df)):
#     # if(count>=5):
#     #     break
#     # else:
#     #     count = count + 1
#     elapsed_time = time.time()
#     if elapsed_time - start_time > TIME_LIMIT:
#             print(f"\nTime limit of {TIME_LIMIT} seconds exceeded. Aborting loop at index {idx}.")
#             print(f'Number of successfully run images - {idx}')
#             break
#     image_path = Path(row['full_image_path'])
#     if not image_path.exists():
#         continue

#     try:
#         image = Image.open(image_path).convert("RGB")
#     except:
#         continue

#     question = row['question']
#     actual_answer = normalize_answer(row['answer'])

#     try:

#         # Prepare inputs using the processor
#         # Get text prompt from chat template
#         prompt = f"<image>\nBased on the image, answer the following question with a single word. Understand this properly for once, give the adjective describing the image according to the question as the first word of the answer and out only one word. Don't use short forms or abbreviations for the answer. Output the full word.  The adjective should not be at the end of the answer.It should be at the start. Question: {question} Answer:"
#         inputs = processor(text=prompt, image=image, return_tensors="pt").to(model.device)


#         # Generate prediction
#         with torch.no_grad():
#             output = model.generate(**inputs, max_new_tokens=10)
#             # Decode raw output
#             pred_full = processor.decode(output[0], skip_special_tokens=True)
#             if "Answer:" in pred_full:
#                 answer_portion = pred_full.split("Answer:")[-1]
#             else:
#                 answer_portion = pred_full
#             words = answer_portion.split()
#             if words:
#                 # words = [t.rstrip('.,;:!?') for t in words if t.strip()]
#                 # pred = "".join(words)
#                 if(words[0] == 'The'):
#                     pred = words[len(words)-1].rstrip('\'.,;:!?')
#                 else:
#                     pred = words[0].rstrip('\'.,;:!?')
#             else:
#                 pred = "" # Handle empty prediction
#             pred = normalize_answer(pred)
            

#     except Exception as e:
#         print(f"Error at index {idx}: {e}")
#         continue

#     if not pred:
#         continue

#     all_preds_GRANITE.append(pred)
#     all_actuals_GRANITE.append(actual_answer)

#     if pred == actual_answer:
#         correct_GRANITE += 1
#     total_GRANITE += 1
#     #print(f"Q: {question} | GT: {actual_answer} | Pred: {pred}")
#     if idx % 1000 == 0:
#         print(f"Processed {idx}. Correct: {correct_GRANITE}/{total_GRANITE}")

In [9]:
from torchvision import transforms
import time
all_preds_GRANITE = []
all_actuals_GRANITE = []
correct_GRANITE = 0
total_GRANITE = 0
results = []

In [10]:
import os
os.makedirs("/kaggle/working/resultCSV", exist_ok=True)
pred_path = '/kaggle/working/resultCSV/predictions.csv'

In [11]:
from tqdm import tqdm
# count  = 0
start_time = time.time()
TIME_LIMIT = 42000
with torch.no_grad():
    for idx, row in tqdm(test_df.iterrows(), total=len(test_df)):
        elapsed_time = time.time()
        if elapsed_time - start_time > TIME_LIMIT:
            print(f"\nTime limit of {TIME_LIMIT} seconds exceeded. Aborting loop at index {idx}.")
            print(f'Number of successfully run images - {idx}')
            break
        image_path = row["full_image_path"]
        question = row['question']
        gt_answer = normalize_answer(row['answer'])
        try:
            image = Image.open(image_path).convert("RGB")
        except FileNotFoundError:
            print(f"Warning: Image not found at {image_path}, skipping row {idx}")
            # count += 1
            # if count == 5:
            #     break
            continue
        except Exception as e:
            print(f"Error loading image {image_path}: {e}, skipping row {idx}")
            # count += 1
            # if count == 5:
            #     break
            continue
    
        prompt = f"Based on the image, answer the following question with a single word. Question: {question} Answer:"
        conversation = [
            {
                "role": "user",
                "content": [
                    {"type": "image", "image": image},
                    {"type": "text", "text": prompt},
                ],
            },
        ]
        # token = processor.tokenizer.image_token
        # print(token)
        # prompt = f"<image>\nBased on the image, answer the following question with a single word. Strictly give a single word as an answer, no sentences, just a single word. Your generated output should only be a single word which is the answer. If it's more than word, you will be penalized. Start the answer with the adjective. I want the start of the answer as the adjective otherwise you will be heavily penalized. Understand this properly for once, give the adjective describing the image according to the question as the first word of the answer and out only one word. Understand this otherwise you will be penalized heavily. Don't use short forms or abbreviations for the answer. Output the full word. Understand this carefully. The adjective should not be at the end of the answer. It should be at the start.  Question: {q} Answer:"
        # inputs = processor(text=prompt, image=image, return_tensors="pt")

        inputs = processor.apply_chat_template(
            conversation,
            add_generation_prompt=True,
            tokenize=True,
            return_dict=True,
            return_tensors="pt"
            )
        inputs = {k: v.to("cuda") for k, v in inputs.items()}
        for k in inputs:
            if k != "input_ids" and k != "attention_mask":  # Keep input_ids and attention_mask as long/int
                inputs[k] = inputs[k].to(torch.float16)
        output = model.generate(**inputs, max_new_tokens=10)
        pred_full = processor.decode(output[0], skip_special_tokens=True)

        if "Answer:" in pred_full:
            answer_portion = pred_full.split("Answer:\n<|assistant|>")[-1]
        else:
            answer_portion = pred_full
        words = answer_portion.split()
        if words:
            words = [t.rstrip('.,;:!?') for t in words if t.strip()]
            pred = "".join(words)
            # pred = words[0].rstrip('\'.,;:!?')
        else:
            pred = "" # Handle empty prediction

        # count +=1
        # Normalize, strip, and lowercase
        # pred = normalize_answer(pred).strip().lower()
        # gt_answer = normalize_answer(gt_answer).strip().lower() if pd.notna(gt_answer) else ""
        

        all_preds_GRANITE.append(pred)
        all_actuals_GRANITE.append(gt_answer.strip() if pd.notna(gt_answer) else "")
        results.append({
        "img_path": image_path,
        "question": question,
        "true_answer": str(gt_answer).lower(),
        "predicted_answer": str(pred).lower()
        })
        # print(f"\nIdx: {idx}")
        # print(f"Original Question: {question}")
        # print(f"Full Prediction: '{pred_full}'")
        # print(f"Processed Prediction (Single Word Attempt): '{pred}'")
        # print(f"Answer ground truth: '{gt_answer}'")
        #print("count: ", count)
        # if (count == 100): # Limiting to 100 for the example
        #     print("\nReached count limit (100). Breaking loop.")
        #     break
        if (idx + 1) % 1000 == 0 or (idx + 1) == len(test_df):
        # Convert to DataFrame
            chunk_df = pd.DataFrame(results)
    
            # Save predictions
            if not os.path.exists(pred_path):
                chunk_df.to_csv(pred_path, mode='w', index=False, header=True)
            else:
                chunk_df.to_csv(pred_path, mode='a', index=False, header=False)
    
            print(f"Checkpoint saved at index {idx + 1}")
            # Reset for next chunk
            results = []

print("length of predictions", len(all_preds_GRANITE))

  5%|▍         | 1000/20001 [49:19<15:35:50,  2.96s/it]

Checkpoint saved at index 1000


 10%|▉         | 2000/20001 [1:38:49<14:50:11,  2.97s/it]

Checkpoint saved at index 2000


 15%|█▍        | 3000/20001 [2:27:49<14:12:15,  3.01s/it]

...

Checkpoint saved at index 13000


 70%|██████▉   | 14000/20001 [11:31:21<5:02:58,  3.03s/it]

Checkpoint saved at index 14000


 71%|███████   | 14177/20001 [11:40:02<4:47:35,  2.96s/it]


Time limit of 42000 seconds exceeded. Aborting loop at index 14177.
Number of successfully run images - 14177
length of predictions 14177


In [12]:
# Normalize case
preds = pd.read_csv('/kaggle/working/resultCSV/predictions.csv')
preds = preds.dropna()
# preds = preds.iloc[1:].reset_index(drop=True)
all_preds_GRANITE = preds['predicted_answer'].tolist()
all_actuals_GRANITE = preds['true_answer'].tolist()

In [13]:


# Normalize case
preds_l = [p.lower() for p in all_preds_GRANITE]
refs_l = [r.lower() for r in all_actuals_GRANITE]

# Compute exact-match binary metrics
y_pred_bin = [int(p == r) for p, r in zip(preds_l, refs_l)]
y_true_bin = [1] * len(all_actuals_GRANITE)

acc = accuracy_score(y_true_bin, y_pred_bin)
prec, rec, f1, _ = precision_recall_fscore_support(
    y_true_bin, y_pred_bin, average="binary", zero_division=0
)

# Print metrics
print(f"Exact-match Accuracy: {acc:.3f}")
print(f"Exact-match Precision: {prec:.3f}")
print(f"Exact-match Recall:    {rec:.3f}")
print(f"Exact-match F1:        {f1:.3f}\n")

# Save predictions and ground truths to CSV
# pred_ref_df = pd.DataFrame({
#     "Prediction": predictions,
#     "Ground_Truth": refs
# })
# df.to_csv('prediction_output.csv', index=False)

Exact-match Accuracy: 0.530
Exact-match Precision: 1.000
Exact-match Recall:    0.530
Exact-match F1:        0.692



In [14]:
from bert_score import score

P, R, F1 = score(preds_l, refs_l, lang="en", verbose=True, rescale_with_baseline = True)
print(f"BERTScore - Precision: {P.mean():.4f}, Recall: {R.mean():.4f}, F1: {F1.mean():.4f}")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/33 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/219 [00:00<?, ?it/s]

done in 6.46 seconds, 2165.73 sentences/sec
BERTScore - Precision: 0.8536, Recall: 0.8762, F1: 0.8626


In [15]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
rouge1_scores = []
rougeL_scores = []

for pred, ref in zip(preds_l, refs_l):
    scores = scorer.score(ref, pred)
    rouge1_scores.append(scores["rouge1"].fmeasure)
    rougeL_scores.append(scores["rougeL"].fmeasure)

print(f"ROUGE Scores:")
print(f"ROUGE-1 F1: {np.mean(rouge1_scores):.4f}")
print(f"ROUGE-L F1: {np.mean(rougeL_scores):.4f}")

ROUGE Scores:
ROUGE-1 F1: 0.5442
ROUGE-L F1: 0.5442


In [16]:
lev_dists = [Levenshtein.distance(p, r) for p, r in zip(preds_l, refs_l)]
avg_lev = np.mean(lev_dists)
print(f"Levenshtein Distance:")
print(f"Average Edit Distance: {avg_lev:.2f}")

Levenshtein Distance:
Average Edit Distance: 3.03


In [17]:
# 5. Sentence Embedding Cosine Similarity (memory-efficient pairwise)
model = SentenceTransformer('all-MiniLM-L6-v2')
pred_embeds = model.encode(preds_l, convert_to_tensor=True, device='cuda')
ref_embeds = model.encode(refs_l, convert_to_tensor=True, device='cuda')

cos_sims = [
    util.cos_sim(pred_embeds[i], ref_embeds[i]).item()
    for i in range(len(preds_l))
]

print(f"Sentence-BERT Cosine Similarity:")
print(f"Average Cosine Similarity: {np.mean(cos_sims):.4f}")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/438 [00:00<?, ?it/s]

Batches:   0%|          | 0/438 [00:00<?, ?it/s]

Sentence-BERT Cosine Similarity:
Average Cosine Similarity: 0.7745
